In [92]:
# Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [93]:
# Setting the current working directory
import os; os.chdir('/content/drive/MyDrive/AI_ML/Projects/Capstone-NLP')

### Importing the Packages

In [94]:
# Importing the libraries
# load packages
import os

import flask
app = flask.Flask(__name__)
from flask import Flask, render_template, request


In [95]:
!mkdir templates -p

In [96]:
!ls

chatbot.ipynb
glove.6B.200d.txt
glove.6B.50d.txt
IHMStefanini_industrial_safety_and_health_database_with_accidents_description.csv
log
main_chatbot.ipynb
main.py.ipynb
model.h5
model.json
model.png
prediction.html
sarcasm_detector.h5
search_page.html
SMSSpamCollection
templates
tokenizer.pickle
word2vec.glove.6B.50d.txt
Word2Vec.ipynb
WordCNN.ipynb


In [97]:
%%writefile templates/prediction.html

<!DOCTYPE html>
<div class="header">
  <h1>Accident Level</h1>
</div>

<div id="search-info">
  <h2>There is a {{ prediction }}% chance your description is of accident level {{  }}.</h2>
</div>



<style>
.header {
  font-family: 'Proxima Novas', sans-serif;
  padding: 10px;
  text-align: center;
  background: #0077cc;
  color: #ffffff;
  font-size: 18px;
}

#search-info {
  text-align: center;
  font-family: 'Proxima Nova', sans-serif;
  color: #0077cc;
  font-size: 18px;
  font-weight: 100;
  padding: 20px;
  margin: -30px -30px -10px -30px;
}

</style>


Overwriting templates/prediction.html


In [98]:
%%writefile templates/search_page.html

<!DOCTYPE html>
<div class="header">
  <h1>Accident Level ?</h1>
</div>

<div id="search-info">
  <h2>Accident Level predictor using deep learning</h2>
</div>

<div class="form-style-8">
  <form id="search_form" action="/" method="POST">
    <input type="text" name="text_entered" placeholder="Enter text received" />
    <input type="submit" value="Get Accident Level prediction!" id="SubmitButton" />
  </form>
</div>

<script src="https://ajax.googleapis.com/ajax/libs/jquery/1.11.1/jquery.min.js"></script>
<script src="https://maxcdn.bootstrapcdn.com/bootstrap/3.2.0/js/bootstrap.min.js"></script>
<script>

window.onload = function() {
  jQuery(document).keypress(function(e) {
    if (e.keyCode == 13) {
      e.preventDefault();
      jQuery('#SubmitButton').click();
    }
  });
}

</script>



<style>
.header {
  font-family: 'Proxima Novas', sans-serif;
  padding: 10px;
  text-align: center;
  background: #0077cc;
  color: #ffffff;
  font-size: 18px;
}

#search-info {
  text-align: center;
  font-family: 'Proxima Nova', sans-serif;
  color: #0077cc;
  font-size: 18px;
  font-weight: 100;
  padding: 20px;
  margin: -30px -30px -10px -30px;
}

<link href='http://fonts.googleapis.com/css?family=Open+Sans+Condensed:300' rel='stylesheet' type='text/css'><style type="text/css">.form-style-8 {
  font-family: 'Proxima Novas', sans-serif;
  width: 500px;
  padding: 30px;
  background: #FFFFFF;
  margin: 50px auto;
  box-shadow: 0px 0px 15px rgba(0, 0, 0, 0.22);
  -moz-box-shadow: 0px 0px 15px rgba(0, 0, 0, 0.22);
  -webkit-box-shadow: 0px 0px 15px rgba(0, 0, 0, 0.22);
}

.form-style-8 input[type="text"],
.form-style-8 textarea,
.form-style-8 select {
  box-sizing: border-box;
  -webkit-box-sizing: border-box;
  -moz-box-sizing: border-box;
  outline: none;
  display: block;
  width: 100%;
  padding: 10px;
  border: none;
  border-bottom: 1px solid #ddd;
  background: transparent;
  margin-bottom: 10px;
  font: 16px 'Lucida Sans', sans-serif;
  height: 45px;
}

.form-style-8 textarea {
  resize: none;
  overflow: hidden;
}

.form-style-8 input[type="button"],
.form-style-8 input[type="submit"] {
  -moz-box-shadow: inset 0px 1px 0px 0px #f6f6f6;
  -webkit-box-shadow: inset 0px 1px 0px 0px #f6f6f6;
  box-shadow: inset 0px 1px 0px 0px #f6f6f6;
  background-color: #c21515;
  border: 1px solid #f6f6f6;
  display: inline-block;
  cursor: pointer;
  color: #ffffff;
  font-family: 'Lucida Sans', sans-serif;
  font-size: 14px;
  padding: 8px 18px;
  text-decoration: none;
}

.form-style-8 input[type="button"]:hover,
.form-style-8 input[type="submit"]:hover {
  background: linear-gradient(to bottom, #c21515 50%, #f6f6f6 100%);
  background-color: #c21515;
}
</style>


Overwriting templates/search_page.html


In [99]:
!pip install flask-ngrok

In [100]:
#load model preprocessing
import numpy as np
import pandas as pd
import pickle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras.models
from keras.models import model_from_json

In [101]:
# Load tokenizer for preprocessing
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [102]:
# Load pre-trained model into memory
json_file = open('model.json','r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

In [103]:
# Load weights into new model
loaded_model.load_weights("model.h5")

In [104]:
def prepData(text):
    # Convert to array
    textDataArray = [text]
    
    # Convert into list with word ids
    Features = tokenizer.texts_to_sequences(textDataArray)
    Features = pad_sequences(Features, 20, padding='post')
    
    return Features

In [105]:
loaded_model.compile(optimizer="Adam",loss='binary_crossentropy',metrics=['accuracy'])

In [106]:
from flask_ngrok import run_with_ngrok 

app = Flask(__name__) 
run_with_ngrok(app)   

In [107]:
#@app.route("/")
#def home():
#    return "<h1>Running Flask on Google Colab!</h1>"
#    #return render_template('prediction.html', prediction=prediction)

In [108]:
# define a predict function as an endpoint 

@app.route('/', methods=['GET','POST'])
def predict():
    
    #whenever the predict method is called, we're going
    #to input the user entered text into the model
    #and return a prediction
    
    if request.method=='POST':
        textData = request.form.get('text_entered')
        Features = prepData(textData)
        prediction = int((np.asscalar(loaded_model.predict(Features)))*100)
        return render_template('prediction.html', prediction=prediction)
        print(prediction.dtypes)
    
    else:
        return render_template("search_page.html")



In [111]:
text_entered = "The collaborator reports that he was on street 09 holding in his left hand the volumetric balloon, when he slipped and when placing his hand on the ground the volumetric balloon ended up breaking caused a small wound in his left hand."

textData = text_entered
Features = prepData(textData)
prediction = int((np.asscalar(loaded_model.predict(Features)))*100)
prediction

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  """


54

In [ ]:
if __name__ == "__main__":
    app.run()